Import relevant libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dask.distributed import LocalCluster
import dask.dataframe as dd

In [2]:
#cluster = LocalCluster()
#client = cluster.get_client()

In [3]:
#client

In [4]:
data_file = "./mmtd.txt"

In [5]:
id_cols = ['artist_name', 'track_title']

In [6]:
df_mmtd_temp = pd.read_csv(data_file, sep='\t')

In [7]:
df_mmtd_temp.shape

(1090726, 291)

In [8]:
df_mmtd_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090726 entries, 0 to 1090725
Columns: 291 entries, tweet_id to lang_zu
dtypes: float64(264), int64(7), object(20)
memory usage: 2.4+ GB


In [9]:
df_mmtd_temp.columns

Index(['tweet_id', 'tweet_tweetId', 'tweet_userId', 'tweet_artistId',
       'tweet_trackId', 'tweet_datetime', 'tweet_unixtime', 'tweet_weekday',
       'tweet_longitude', 'tweet_latitude',
       ...
       'lang_wuu', 'lang_xal', 'lang_xh', 'lang_yao', 'lang_yap', 'lang_yo',
       'lang_yue', 'lang_za', 'lang_zh', 'lang_zu'],
      dtype='object', length=291)

In [10]:
df_unique_tracks = df_mmtd_temp[id_cols].drop_duplicates()

In [11]:
df_unique_tracks.shape

(127173, 2)

In [12]:
df_mmtd_temp.shape

(1090726, 291)

In [13]:
df_mmtd_temp.columns[:45]

Index(['tweet_id', 'tweet_tweetId', 'tweet_userId', 'tweet_artistId',
       'tweet_trackId', 'tweet_datetime', 'tweet_unixtime', 'tweet_weekday',
       'tweet_longitude', 'tweet_latitude', 'artist_mbid', 'artist_name',
       'track_title', 'track_7digitalId', 'amazon_asin', 'amazon_albumAsin',
       'country', 'state', 'county', 'city', 'postalCode', 'street',
       'timezone', 'countryName', 'isoAlpha3', 'fipsCode', 'continent',
       'continentName', 'capital', 'areaInSqKm', 'population', 'currencyCode',
       'languages', 'west', 'north', 'east', 'south', 'lang_aa', 'lang_ady',
       'lang_af', 'lang_ak', 'lang_am', 'lang_ar', 'lang_arc', 'lang_as'],
      dtype='object')

In [14]:
df_mmtd_temp["tweet_trackId"].nunique()

134199

In [15]:
df_unique_Tracks_2 = df_mmtd_temp[["artist_name", "track_title", "tweet_trackId"]].drop_duplicates()

In [16]:
df_unique_Tracks_2.shape

(134199, 3)

In [17]:
df_unique_Tracks_2[df_unique_Tracks_2.duplicated(subset=['artist_name','track_title'], keep=False)].sort_values(["artist_name"]).head(20)

,artist_name,track_title,tweet_trackId
1014002,'N Sync,Bye Bye Bye,14289507
53616,'N Sync,Bye Bye Bye,5270936
708591,'N Sync,Bye Bye Bye,9526530
1072114,(Rank 1,L.E.D. There Be Light,14341015
707949,(Rank 1,Airwave,9594195
204641,(Rank 1,Airwave,6057178
279936,(Rank 1,Wild and Perfect Day,6057381
1063104,(Rank 1,Wild and Perfect Day,14341112
305356,(Rank 1,L.E.D. There Be Light,6057284
200215,10cc,I'm Not in Love,12284


In [18]:
!pip install pylast

In [19]:
import pylast

In [20]:
api_key = "a32e2f74379d1ca0a103e25a39338d8b"
api_secret = "40fd7fa49ae684e075f23be69ea4abfe"

In [21]:
genreFilter = ['rnb', 'rap', 'electronic', 'rock', 'newage', 'classical', 'reggae', 'blues', 'country', 'world', 'folk', 'easylistening', 'jazz', 'vocal', 'children\'s', 'punk', 'alternative', 'spokenword', 'pop', 'heavymetal']

In [22]:
genreFilterSet = set(genreFilter)

In [23]:
df_unique_tracks = df_unique_tracks.assign(**dict.fromkeys(genreFilter, 0))

In [35]:
def getGenre(row):
    artist = row['artist_name']
    title = row['track_title']
    try:
        genre = pylast.LastFMNetwork(api_key=api_key, api_secret=api_secret).get_track(title=title, artist=artist).get_top_tags()
    except:
        genre = []
    currGenreSet = set([j[0].get_name() for j in genre])
    songGenres = genreFilterSet.intersection(currGenreSet)
    if len(songGenres) > 0:
            row.__setitem__(list(songGenres), 1)
    if row.name%1000 == 0:
        print(row.name, row[list(songGenres)])
    return row[genreFilter]

In [ ]:
#df_mmtd_temp.npartitions

In [34]:
df_unique_tracks.shape

(127173, 22)

In [36]:
last_network = pylast.LastFMNetwork(api_key=api_key, api_secret=api_secret)

df_unique_tracks.loc[:, genreFilter] = df_unique_tracks.apply(getGenre, axis=1)

0 Series([], Name: 0, dtype: object)
1000 Series([], Name: 1000, dtype: object)
2000 rap    1
rnb    1
Name: 2000, dtype: object
3000 pop    1
Name: 3000, dtype: object
4000 pop     1
rnb     1
jazz    1
Name: 4000, dtype: object
5000 Series([], Name: 5000, dtype: object)
8000 reggae         1
alternative    1
rock           1
pop            1
punk           1
Name: 8000, dtype: object
10000 punk           1
alternative    1
pop            1
rock           1
Name: 10000, dtype: object
11000 blues    1
pop      1
rnb      1
Name: 11000, dtype: object
12000 rap           1
electronic    1
pop           1
rnb           1
Name: 12000, dtype: object
15000 Series([], Name: 15000, dtype: object)
17000 jazz     1
vocal    1
rock     1
blues    1
pop      1
folk     1
Name: 17000, dtype: object
18000 rap    1
Name: 18000, dtype: object
24000 electronic     1
alternative    1
rock           1
jazz           1
Name: 24000, dtype: object
30000 rock    1
Name: 30000, dtype: object
32000 Series([], 

In [40]:
df_unique_tracks.head(8)

,artist_name,track_title,rnb,rap,electronic,rock,newage,classical,reggae,blues,...,folk,easylistening,jazz,vocal,children's,punk,alternative,spokenword,pop,heavymetal
0,Anonymous 4,Agnus dei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Drake,Find Your Love,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Van Morrison,Hey Mr DJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Carly Rae Jepsen,Call Me Maybe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Justin Timberlake,4 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Avant,When It Hurts,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Maria Carey,Love Takes Time,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,Tyga,Far Away,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_unique_tracks.head(10)

,artist_name,track_title,rnb,rap,electronic,rock,newage,classical,reggae,blues,...,folk,easylistening,jazz,vocal,children's,punk,alternative,spokenword,pop,heavymetal
0,Anonymous 4,Agnus dei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Drake,Find Your Love,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Van Morrison,Hey Mr DJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Carly Rae Jepsen,Call Me Maybe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Justin Timberlake,4 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Avant,When It Hurts,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Maria Carey,Love Takes Time,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,Tyga,Far Away,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Of Mice & Men,Purified,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Allman Brothers Band,Stand Back,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
genreFilter

['rnb',
 'rap',
 'electronic',
 'rock',
 'newage',
 'classical',
 'reggae',
 'blues',
 'country',
 'world',
 'folk',
 'easylistening',
 'jazz',
 'vocal',
 "children's",
 'punk',
 'alternative',
 'spokenword',
 'pop',
 'heavymetal']

In [45]:
newGenreFilter = ['genre_' + genre for genre in genreFilter]
newGenreFilter

['genre_rnb',
 'genre_rap',
 'genre_electronic',
 'genre_rock',
 'genre_newage',
 'genre_classical',
 'genre_reggae',
 'genre_blues',
 'genre_country',
 'genre_world',
 'genre_folk',
 'genre_easylistening',
 'genre_jazz',
 'genre_vocal',
 "genre_children's",
 'genre_punk',
 'genre_alternative',
 'genre_spokenword',
 'genre_pop',
 'genre_heavymetal']

In [46]:
genreDict = dict(zip(genreFilter, newGenreFilter))

In [47]:
genreDict

{'rnb': 'genre_rnb',
 'rap': 'genre_rap',
 'electronic': 'genre_electronic',
 'rock': 'genre_rock',
 'newage': 'genre_newage',
 'classical': 'genre_classical',
 'reggae': 'genre_reggae',
 'blues': 'genre_blues',
 'country': 'genre_country',
 'world': 'genre_world',
 'folk': 'genre_folk',
 'easylistening': 'genre_easylistening',
 'jazz': 'genre_jazz',
 'vocal': 'genre_vocal',
 "children's": "genre_children's",
 'punk': 'genre_punk',
 'alternative': 'genre_alternative',
 'spokenword': 'genre_spokenword',
 'pop': 'genre_pop',
 'heavymetal': 'genre_heavymetal'}

In [50]:
df_unique_tracks.rename(columns=genreDict, inplace=True)

In [51]:
df_unique_tracks.head()

,artist_name,track_title,genre_rnb,genre_rap,genre_electronic,genre_rock,genre_newage,genre_classical,genre_reggae,genre_blues,...,genre_folk,genre_easylistening,genre_jazz,genre_vocal,genre_children's,genre_punk,genre_alternative,genre_spokenword,genre_pop,genre_heavymetal
0,Anonymous 4,Agnus dei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Drake,Find Your Love,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Van Morrison,Hey Mr DJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Carly Rae Jepsen,Call Me Maybe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Justin Timberlake,4 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
df_mmtd_temp.shape

(1090726, 291)

In [96]:
df_mmtd_temp.shape

(1090726, 291)

In [97]:
df_unique_tracks.shape

(127173, 22)

In [98]:
df_unique_tracks.head()

,artist_name,track_title,genre_rnb,genre_rap,genre_electronic,genre_rock,genre_newage,genre_classical,genre_reggae,genre_blues,...,genre_folk,genre_easylistening,genre_jazz,genre_vocal,genre_children's,genre_punk,genre_alternative,genre_spokenword,genre_pop,genre_heavymetal
0,Anonymous 4,Agnus dei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Drake,Find Your Love,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Van Morrison,Hey Mr DJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Carly Rae Jepsen,Call Me Maybe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Justin Timberlake,4 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
df_mmtd_with_genre = pd.merge(df_mmtd_temp, df_unique_tracks, on=["artist_name", "track_title"], how="outer")

In [101]:
df_mmtd_with_genre.shape

(1090726, 311)

In [115]:
df_mmtd_with_genre = df_mmtd_with_genre.sort_values(by=["tweet_id"])

In [117]:
df_mmtd_with_genre = df_mmtd_with_genre.reset_index()

In [122]:
df_mmtd_with_genre = df_mmtd_with_genre.drop(["index"], axis=1)

In [123]:
df_mmtd_with_genre.head()

,tweet_id,tweet_tweetId,tweet_userId,tweet_artistId,tweet_trackId,tweet_datetime,tweet_unixtime,tweet_weekday,tweet_longitude,tweet_latitude,...,genre_folk,genre_easylistening,genre_jazz,genre_vocal,genre_children's,genre_punk,genre_alternative,genre_spokenword,genre_pop,genre_heavymetal
0,1,167408118735699971,24842995,25040,420585,2012-02-09 00:43:00,1328744580,3,0.126573,52.1976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,167408251737088000,77147006,122992,2106213,2012-02-09 00:43:32,1328744612,3,-80.199900,36.0349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,185189790390558720,174626103,468091,8030020,2012-03-29 02:21:01,1332980461,3,-0.142822,51.5207,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,229940340692500480,538669914,67904,1210996,2012-07-30 14:03:43,1343649823,0,2.334750,48.9009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,167408424307523584,245555020,223140,4019560,2012-02-09 00:44:13,1328744653,3,-47.818000,-21.2134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
df_mmtd_temp.head()

,tweet_id,tweet_tweetId,tweet_userId,tweet_artistId,tweet_trackId,tweet_datetime,tweet_unixtime,tweet_weekday,tweet_longitude,tweet_latitude,...,lang_wuu,lang_xal,lang_xh,lang_yao,lang_yap,lang_yo,lang_yue,lang_za,lang_zh,lang_zu
0,1,167408118735699971,24842995,25040,420585,2012-02-09 00:43:00,1328744580,3,0.126573,52.1976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,167408251737088000,77147006,122992,2106213,2012-02-09 00:43:32,1328744612,3,-80.199900,36.0349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,185189790390558720,174626103,468091,8030020,2012-03-29 02:21:01,1332980461,3,-0.142822,51.5207,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,229940340692500480,538669914,67904,1210996,2012-07-30 14:03:43,1343649823,0,2.334750,48.9009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,167408424307523584,245555020,223140,4019560,2012-02-09 00:44:13,1328744653,3,-47.818000,-21.2134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
df_mmtd_with_genre.to_csv("mmtd_with_genre.csv", index=False)

In [125]:
df_mmtd_with_genre.shape

(1090726, 311)

In [ ]:
# Code ends here.  What follows is scratch code.

In [66]:
"""
last_network = pylast.LastFMNetwork(api_key=api_key, api_secret=api_secret)

for idx, row in df_unique_tracks.iterrows():
    artist = row['artist_name']
    title = row['track_title']
    try:
        genre = last_network.get_track(title=title, artist=artist).get_top_tags(limit=10)
    except:
        genre = []
    currGenreSet = set([j[0].get_name() for j in genre])
    songGenres = genreFilterSet.intersection(currGenreSet)
    row[list(songGenres)] = True
    if idx%1000 == 0:
        print(idx, songGenres, row[list(songGenres)])
"""

0 set() Series([], Name: 0, dtype: object)
1000 set() Series([], Name: 1000, dtype: object)
2000 {'rnb', 'rap'} rnb    True
rap    True
Name: 2000, dtype: object
3000 {'pop'} pop    True
Name: 3000, dtype: object
4000 {'rnb'} rnb    True
Name: 4000, dtype: object
5000 set() Series([], Name: 5000, dtype: object)
8000 {'rock', 'reggae', 'pop'} rock      True
reggae    True
pop       True
Name: 8000, dtype: object
10000 {'rock', 'alternative', 'pop'} rock           True
alternative    True
pop            True
Name: 10000, dtype: object
11000 {'rnb'} rnb    True
Name: 11000, dtype: object
12000 {'rap', 'pop'} rap    True
pop    True
Name: 12000, dtype: object
15000 set() Series([], Name: 15000, dtype: object)
17000 {'rock', 'pop'} rock    True
pop     True
Name: 17000, dtype: object
18000 {'rap'} rap    True
Name: 18000, dtype: object
24000 {'alternative', 'electronic'} alternative    True
electronic     True
Name: 24000, dtype: object
30000 {'rock'} rock    True
Name: 30000, dtype: object

In [82]:
df_mmtd_temp['artist_name'].nunique()

24549

In [85]:
df_unique_tracks['rock'].nunique()

1

In [ ]:
#df_mmtd_temp.compute()

In [ ]:
# Plot the number of tweets per genre
df_mmtd[genreFilter].sum().plot(kind='bar', title='Number of Tweets per Genre', rot=45)
plt.xlabel('Genre')
plt.ylabel('Number of Tweets')
plt.show()

In [ ]:
# plot the number of Tweets per country
df_mmtd_10k['countryName'].value_counts().plot(kind='bar', title='Number of Tweets per Country')
plt.xlabel('Country')
plt.ylabel('Number of Tweets')
plt.show()

In [ ]:
# Spatio-temporal Statistical Aspects

sampleCountry = ['Brazil', 'France', 'Indonesia', 'Malaysia', 'United States']
sample_dow = [4, 6, 0, 2, 1]
sample_time = [0, 5, 12, 17, 21]

# Find the number of tweets per genre for each combination of country, dow and time

df_st_decomposition = pd.DataFrame(columns=['countryName', 'dow', 'hour'] + genreFilter)

for country, dow in zip(sampleCountry, sample_dow):
  for time in sample_time:
    df_st_decomposition.loc[len(df_st_decomposition)] = [country, dow, time] + list(df_mmtd_10k[(df_mmtd_10k['countryName'] == country) & (df_mmtd_10k['dow'] == dow) & (df_mmtd_10k['hour'] == time)][genreFilter].sum())
df_st_decomposition

In [127]:
df_unique_tracks.head()

,artist_name,track_title,genre_rnb,genre_rap,genre_electronic,genre_rock,genre_newage,genre_classical,genre_reggae,genre_blues,...,genre_folk,genre_easylistening,genre_jazz,genre_vocal,genre_children's,genre_punk,genre_alternative,genre_spokenword,genre_pop,genre_heavymetal
0,Anonymous 4,Agnus dei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Drake,Find Your Love,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Van Morrison,Hey Mr DJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Carly Rae Jepsen,Call Me Maybe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Justin Timberlake,4 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
df_unique_tracks.loc[58]

artist_name                 Taylor Swift
track_title            Should've Said No
genre_rnb                            0.0
genre_rap                            0.0
genre_electronic                     0.0
genre_rock                           1.0
genre_newage                         0.0
genre_classical                      0.0
genre_reggae                         0.0
genre_blues                          0.0
genre_country                        1.0
genre_world                          0.0
genre_folk                           0.0
genre_easylistening                  0.0
genre_jazz                           0.0
genre_vocal                          0.0
genre_children's                     0.0
genre_punk                           0.0
genre_alternative                    0.0
genre_spokenword                     0.0
genre_pop                            1.0
genre_heavymetal                     0.0
Name: 58, dtype: object

In [138]:
df_unique_tracks.loc[58][df_unique_tracks.loc[58]==1]

genre_rock       1.0
genre_country    1.0
genre_pop        1.0
Name: 58, dtype: object

In [139]:
df_unique_tracks.head()

,artist_name,track_title,genre_rnb,genre_rap,genre_electronic,genre_rock,genre_newage,genre_classical,genre_reggae,genre_blues,...,genre_folk,genre_easylistening,genre_jazz,genre_vocal,genre_children's,genre_punk,genre_alternative,genre_spokenword,genre_pop,genre_heavymetal
0,Anonymous 4,Agnus dei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Drake,Find Your Love,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Van Morrison,Hey Mr DJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Carly Rae Jepsen,Call Me Maybe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Justin Timberlake,4 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
df_unique_tracks.head()

NameError: name 'df_unique_tracks' is not defined